In [88]:
import os
import csv
import json

import pandas as pd
from sklearn.model_selection import train_test_split

SEED = 42

In [89]:
data_path = '../../data/google_translate.jsonl'
with open(data_path, 'r', encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

In [90]:
data[0]

{'message_tree_id': '001199c7-7135-4e64-bee9-48ea862243b4',
 'tree_state': 'ready_for_export',
 'prompt': {'message_id': '001199c7-7135-4e64-bee9-48ea862243b4',
  'user_id': '42398a03-04b3-4028-9d97-1444708e91cd',
  'created_date': '2023-02-04T18:54:01.048716+00:00',
  'text': 'Do you have any information about the Commodore 64?',
  'role': 'prompter',
  'lang': 'en',
  'review_count': 3,
  'review_result': True,
  'deleted': False,
  'synthetic': False,
  'emojis': {'+1': 8, '_skip_reply': 2},
  'replies': [{'message_id': '314ea243-638d-48d6-9f3d-8edf80f85ec2',
    'parent_id': '001199c7-7135-4e64-bee9-48ea862243b4',
    'user_id': '00bbfff8-0e6a-4e46-93cc-65785d78c8bc',
    'created_date': '2023-02-05T00:38:59.927142+00:00',
    'text': "Of course. The Commodore 64 is a 8-bit home computer that was released by Commodore International in August 1982. It is one of the most successful home computers of all time, with over 17 million units sold. It featured 64 kilobytes of RAM, color gra

In [91]:
def tree_to_conversations(tree, split="train"):
    # get pairs for root prompt and replies
    conversations = message_to_conversations(tree["prompt"], '', split=split)
    return conversations


def message_to_conversations(message, previous_conversation="", split="train"):
    conversations = []
    current_message =  "ASISTENT" if message['role'] == "assistant" else "UPORABNIK" + ": " + message['translation']
    current_conversation = previous_conversation + current_message
    
    # check if we have any replies
    if 'replies' in message:
        rank = 0
        for reply in message['replies']:
            if "translation" not in reply:
                continue
            # if reply['role'] == 'prompter':
            #     input_text = reply['translation']
            #     continue
            if 'rank' in reply:
                rank = reply['rank']
            else:
                rank += 1
            
            reply_role = "ASISTENT" if reply['role'] == "assistant" else "UPORABNIK"
            reply_message = reply_role + ": " + reply['translation']
            
            # conversation with start at current prompt
            conversations.append((current_message, reply_message, rank, split))
            
            # conversations extended with the current prompt
            conversations.append((current_conversation, reply_message, rank, split))
            
            # conversations ....
            conversations.extend(message_to_conversations(reply, current_conversation, split))
    return conversations

In [92]:
tree_to_conversations(data[1])

[('UPORABNIK: Metoda DMAIC v akciji\nProduktna skupina opaža, da stopnja odliva strank (stopnja, pri kateri stranke prenehajo poslovati z vami) narašča. Da bi preprečili napredovanje trenda, se uporabi metodologija Six Sigma DMAIC za prepoznavanje problema in razvoj rešitve.\n\nOpredelitev (»definiraj«): Stopnja odliva strank se je v zadnjih 6 mesecih povečala s 3 % na 7 %.\n\nMerjenje: Ekipa ima veliko informacij o tem, kako potencialne stranke postanejo stranke, ne pa toliko o tem, kaj se zgodi potem, ko nekdo postane stranka. Zato je odločeno analizirati in izmeriti vedenje uporabnikov po nakupu izdelka.\n\nAnaliza (»Analyze«): Po opazovanju vedenja uporabnikov, potem ko postanejo dejanske stranke, ekipa ugotavlja, da se nove stranke težje navadijo na nov uporabniški vmesnik izdelka.\n\nIzboljšati: ekipa se odloči implementirati tok vključevanja novih strank, ki uporabnikom pomaga prepoznati in uporabiti ključne dele izdelka. Pri večjih strankah sodelujte z ekipo za uspeh strank, da

In [93]:
pairs = tree_to_conversations(data[0])

for pair in pairs:
    prompt, reply, rank, split = pair
    print(f"PROMPT: {prompt}")
    print(f"REPLY: {reply}")
    print(f"RANK: {rank}")
    print("====================================="*4)

PROMPT: UPORABNIK: Ali imate kakšne informacije o Commodore 64?
REPLY: ASISTENT: Seveda. Commodore 64 je 8-bitni domači računalnik, ki ga je izdal Commodore International avgusta 1982. Je eden najuspešnejših domačih računalnikov vseh časov, z več kot 17 milijoni prodanih enot. Vseboval je 64 kilobajtov RAM-a, barvno grafiko in zvočni čip SID. Široko se je uporabljal za igre, programiranje in domače aplikacije. Nasledil ga je Commodore 128.

Še vedno obstaja uspešna skupnost navdušencev nad Commodore 64 po vsem svetu, ki še naprej uživajo v igranju in programiranju v sistemu. Več informacij o Commodore 64 najdete na različnih spletnih mestih in forumih, posvečenih sistemu.

Če imate kakršna koli posebna vprašanja o strojni opremi, programski opremi ali programskih jezikih Commodore 64, jih vprašajte.
RANK: 0
PROMPT: UPORABNIK: Ali imate kakšne informacije o Commodore 64?
REPLY: ASISTENT: Seveda. Commodore 64 je 8-bitni domači računalnik, ki ga je izdal Commodore International avgusta 19

### Split the trees in train, validation and test sets

In [94]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=SEED, shuffle=True)
data_val, data_test = train_test_split(data_test, test_size=0.5, random_state=SEED, shuffle=True)

In [95]:
len(data_train), len(data_val), len(data_test)

(6923, 865, 866)

### Get the pairs from the trees

In [96]:
def generate_pairs(data, split="train"):
    pairs = []
    for tree in data:
        pairs += tree_to_conversations(tree, split)
    df = pd.DataFrame(pairs, columns=["prompt", "reply", "rank", "split"])
    return df

In [97]:
pairs_train = generate_pairs(data_train, "train")
pairs_val = generate_pairs(data_val, "val")
pairs_test = generate_pairs(data_test, "test")

In [98]:
len(pairs_train), len(pairs_val), len(pairs_test)

(106926, 13368, 13786)

In [99]:
pairs_train.head()

,prompt,reply,rank,split
0,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Kaj pa igranje lažnivca? Za to igro ...,0,train
1,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Kaj pa igranje lažnivca? Za to igro ...,0,train
2,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Uno: Je klasična igra za otroke in o...,1,train
3,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Uno: Je klasična igra za otroke in o...,1,train
4,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Igrate lahko:\n - 8 noro\n - celina\...,2,train


In [100]:
pairs_val

,prompt,reply,rank,split
0,"UPORABNIK: Prosimo, da vljudno zahtevate ponud...",ASISTENT: Dame in gospodje\n\nKmalu se selim i...,0,val
1,"UPORABNIK: Prosimo, da vljudno zahtevate ponud...",ASISTENT: Dame in gospodje\n\nKmalu se selim i...,0,val
2,ASISTENT,UPORABNIK: Napišite mi ustrezno zadevo za e-po...,1,val
3,"UPORABNIK: Prosimo, da vljudno zahtevate ponud...",UPORABNIK: Napišite mi ustrezno zadevo za e-po...,1,val
4,UPORABNIK: Napišite mi ustrezno zadevo za e-po...,ASISTENT: Ocena stroškov selitev Nürnberg -> A...,0,val
...,...,...,...,...
13363,"UPORABNIK: Poiščite besede esdrújula, ki se za...",UPORABNIK: Odgovor ni pravilen. Besedi enostav...,1,val
13364,UPORABNIK: Odgovor ni pravilen. Besedi enostav...,"ASISTENT: Prav imate, poskušali boste izboljša...",0,val
13365,"UPORABNIK: Poiščite besede esdrújula, ki se za...","ASISTENT: Prav imate, poskušali boste izboljša...",0,val
13366,UPORABNIK: Odgovor ni pravilen. Besedi enostav...,ASISTENT: Za tako napako se vam iskreno opravi...,1,val


In [101]:
pairs_test

,prompt,reply,rank,split
0,UPORABNIK: Vzemimo naključno matriko $n\krat n...,"ASISTENT: Ocenjevanje natančne verjetnosti, da...",0,test
1,UPORABNIK: Vzemimo naključno matriko $n\krat n...,"ASISTENT: Ocenjevanje natančne verjetnosti, da...",0,test
2,ASISTENT,UPORABNIK: Hvala vam! Jasno mi je,1,test
3,UPORABNIK: Vzemimo naključno matriko $n\krat n...,UPORABNIK: Hvala vam! Jasno mi je,1,test
4,UPORABNIK: Hvala vam! Jasno mi je,ASISTENT: Vedno z veseljem pomagamo! Vas zanim...,0,test
...,...,...,...,...
13781,"UPORABNIK: Imam elektronski set bobnov, a vsak...","ASISTENT: Da, obstaja več načinov, kako to pop...",0,test
13782,UPORABNIK: Kadarkoli poskušam postaviti izolac...,"ASISTENT: Težava, da udarec pobegne, je precej...",1,test
13783,"UPORABNIK: Imam elektronski set bobnov, a vsak...","ASISTENT: Težava, da udarec pobegne, je precej...",1,test
13784,ASISTENT,UPORABNIK: Kakšno boljšo strojno opremo bi pri...,1,test


In [102]:
# combine all splits to a single dataframe
pairs = pd.concat([pairs_train, pairs_val, pairs_test], ignore_index=True)
pairs.head()

,prompt,reply,rank,split
0,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Kaj pa igranje lažnivca? Za to igro ...,0,train
1,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Kaj pa igranje lažnivca? Za to igro ...,0,train
2,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Uno: Je klasična igra za otroke in o...,1,train
3,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Uno: Je klasična igra za otroke in o...,1,train
4,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Igrate lahko:\n - 8 noro\n - celina\...,2,train


In [103]:
# drop duplicate rows
pairs = pairs.drop_duplicates(subset=["prompt", "reply"], keep="first")

In [104]:
pairs.to_csv("../../data/context_reply_pairs.csv", index=False, sep=";")

In [106]:
df = pd.read_csv("../../data/context_reply_pairs.csv", sep=";")